In [ ]:
import gcamreader
import numpy as np
import pandas as pd
import pyarrow as pa


In [ ]:
# TODO: once this code is tested on IM3 data, convert to a convenient Python method

### All the files needed to get the CERF-required GCAM-USA outputs

In [ ]:
gcam_scenario = 'NZ_Climate'
gcam_prefix = 'database_02b_'


gcam_db_path = '/Users/thur961/godeeep/GCAM_IRA_SCENARIOS'
gcam_db_file = f'{gcam_prefix}{gcam_scenario}'

query_path = '/Users/thur961/godeeep/godeeep/GCAM-to-CERF/subSetQueries.xml'

generation_query_name = 'elec gen by gen tech and cooling tech (incl cogen)'
heat_rate_query_name = 'elec coeff'
prices_query_name = 'prices by sector'

tech_mapping_file = '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/inst/extdata/gcam-usa/A23.elecS_tech_mapping_cool.csv'

capacity_factor_conventional_file = '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L2233.GlobalTechCapFac_elecS_cool_USA.csv' # (same for all states)
capacity_factor_wind_file = '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L2237.StubTechCapFactor_wind_reeds_USA.csv' # (differ by state)
capacity_factor_pv_file = '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L2238.StubTechCapFactor_PV_reeds_USA.csv' # (differ by state)
capacity_factor_csp_file = '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L2239.StubTechCapFactor_CSP_reeds_USA.csv' # (differ by state)
nrel_renewables_capacity_factor_file = '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/inst/extdata/gcam-usa/NREL_us_re_capacity_factors.csv' # (differ by state)

fixed_operation_and_maintenance_cost_files = [
    '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L2233.GlobalTechOMfixed_elecS_cool_USA.csv',
    '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L2233.GlobalIntTechOMfixed_elecS_cool_USA.csv',
    '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L2241.GlobalTechOMfixed_coal_vintage_USA.csv',
    '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L2241.GlobalTechOMfixed_elec_coalret_USA.csv',
]

variable_operation_and_maintenance_cost_files = [
    '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L2233.GlobalTechOMvar_elecS_cool_USA.csv',
    '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L2233.GlobalIntTechOMvar_elecS_cool_USA.csv',
    '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L2241.GlobalTechOMvar_coal_vintage_USA.csv',
    '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L2241.GlobalTechOMvar_elec_coalret_USA.csv',
]

carbon_content_files = [
    '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L202.CarbonCoef.csv',
    '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L222.CarbonCoef_en_USA.csv',
    '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L2261.CarbonCoef_bio_USA.csv',
]

carbon_capture_file = '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L2233.GlobalTechCapture_elec_cool.csv'

lifetime_files = [
    '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L2233.GlobalTechLifetime_elecS_cool_USA.csv',
    '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L2233.GlobalIntTechLifetime_elecS_cool_USA.csv',
]

lifetime_parameter_files = [
    '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L2244.StubTechSCurve_nuc_gen2_USA.csv',
    '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L2233.StubTechSCurve_elecS_cool_USA.csv',
    '/Users/thur961/godeeep/gcam-central-net-zero/GCAMUSA_GODEEEP/input/gcamdata/outputs/L2233.GlobalTechSCurve_elecS_cool_USA.csv',
]


### Conversions and mappings

In [ ]:
# lifted from gcamextractor
EXAJOULES_TO_GIGAWATTS = 277.77777777778 * 1000 / 8760
BRITISH_THERMAL_UNITS_PER_KILOWATT_HOUR = 3412.14
GIGAJOULES_TO_MEGA_BRITISH_THERMAL_UNITS = 0.947817120
ATOMIC_WEIGHT_OF_CO2_PER_ATOMIC_WEIGHT_OF_CARBON = 44 / 12
TONS_PER_KG = 1e-3
MEGAWATT_HOURS_PER_GIGAJOULE = 0.277777778
MEGA_BRITISH_THERMAL_UNITS_PER_MEGAWATT_HOUR = 3.412e6
REGION_TO_STATES = {
    "Alaska grid": ["AK"],
    "California grid": ["CA"],
    "Central East grid": ["IN", "KY", "MI", "OH", "WV"],
    "Central Northeast grid": ["IL", "MO", "WI"],
    "Central Northwest grid": ["IA", "MN", "ND", "NE", "SD"],
    "Central Southwest grid": ["KS", "OK"],
    "Florida grid": ["FL"],
    "Hawaii grid": ["HI"],
    "Mid-Atlantic grid": ["DC", "DE", "MD", "NJ", "PA"],
    "New England grid": ["CT", "MA", "ME", "NH", "RI", "VT"],
    "New York grid": ["NY"],
    "Northwest grid": ["ID", "MT", "NV", "OR", "UT", "WA"],
    "Southeast grid": ["AL", "AR", "GA", "LA", "MS", "NC", "SC", "TN", "VA"],
    "Southwest grid": ["AZ", "CO", "NM", "WY"],
    "Texas grid": ["TX"],
}
STATE_ABBREVIATIONS_TO_NAMES = {
    'AB': 'alberta', 'AK': 'alaska', 'AL': 'alabama', 'AR': 'arkansas', 'AZ': 'arizona',
    'CA': 'california', 'BC': 'british_columbia', 'MX': 'mexico',
    'CO': 'colorado', 'CT': 'connecticut', 'DC': 'district_of_columbia', 'DE': 'delaware',
    'FL': 'florida', 'GA': 'georgia', 'HI': 'hawaii', 'IA': 'iowa', 'ID': 'idaho', 'IL': 'illinois',
    'IN': 'indiana', 'KS': 'kansas', 'KY': 'kentucky', 'LA': 'louisiana', 'MA': 'massachusetts',
    'MD': 'maryland', 'ME': 'maine', 'MI': 'michigan', 'MN': 'minnesota', 'MO': 'missouri',
    'MS': 'mississippi', 'MT': 'montana', 'NC': 'north_carolina', 'ND': 'north_dakota',
    'NE': 'nebraska', 'NH': 'new_hampshire', 'NJ': 'new_jersey', 'NM': 'new_mexico', 'NV': 'nevada',
    'NY': 'new_york', 'OH': 'ohio', 'OK': 'oklahoma', 'OR': 'oregon', 'PA': 'pennsylvania',
    'RI': 'rhode_island', 'SC': 'south_carolina', 'SD': 'south_dakota', 'TN': 'tennessee',
    'TX': 'texas', 'UT': 'utah', 'VA': 'virginia', 'VT': 'vermont', 'WA': 'washington',
    'WI': 'wisconsin', 'WV': 'west_virginia', 'WY': 'wyoming'
}
ALL_STATES = sorted([s for states in REGION_TO_STATES.values() for s in states])

### Create the connection to the GCAM-USA output database

In [ ]:
conn = gcamreader.LocalDBConn(gcam_db_path, gcam_db_file)


### List of queries available in the XML query file

In [ ]:
queries = gcamreader.parse_batch_query(query_path)


### Helper methods

In [ ]:
def get_query_by_name(queries, name):
    return next((x for x in queries if x.title == name), None)


In [ ]:
def deflate_gdp(year, base_year):
    # this was lifted from gcamextractor
    start_year = 1929
    factors = np.array([
        9.896, 9.535, 8.555, 7.553, 7.345, 7.749, 7.908, 8.001, 8.347,
        8.109, 8.033, 8.131, 8.68, 9.369, 9.795, 10.027, 10.288, 11.618,
        12.887, 13.605, 13.581, 13.745, 14.716, 14.972, 15.157, 15.298,
        15.559, 16.091, 16.625, 17.001, 17.237, 17.476, 17.669, 17.886,
        18.088, 18.366, 18.702, 19.227, 19.786, 20.627, 21.642, 22.784,
        23.941, 24.978, 26.337, 28.703, 31.361, 33.083, 35.135, 37.602,
        40.706, 44.377, 48.52, 51.53, 53.565, 55.466, 57.24, 58.395,
        59.885, 61.982, 64.392, 66.773, 68.996, 70.569, 72.248, 73.785,
        75.324, 76.699, 78.012, 78.859, 80.065, 81.887, 83.754, 85.039,
        86.735, 89.12, 91.988, 94.814, 97.337, 99.246, 100, 101.221,
        103.311, 105.214, 106.913, 108.828, 109.998, 111.445, 113.545,
        116.311, 118.339
    ])
    return factors[year - start_year] / factors[base_year - start_year]


In [ ]:
def grid_region_to_state(df) -> pd.DataFrame:
    def region_to_list(r):
        if r in REGION_TO_STATES:
            return REGION_TO_STATES[r]
        else:
            return [r]
    df['region'] = df.region.apply(region_to_list)
    df = df.explode('region', ignore_index=True)
    return df


In [ ]:
def get_escalation_rate(df, column, groupby, name, axis='year') -> pd.DataFrame:
    escalation_rate = df.copy()
    escalation_rate[axis] = escalation_rate[axis] - escalation_rate.groupby(groupby)[axis].diff()
    escalation_rate = escalation_rate.rename(columns={column: name})
    escalation_rate = escalation_rate.merge(df, how='left', on=[axis, *groupby])
    escalation_rate[name] = (escalation_rate[name] - escalation_rate[column]) / escalation_rate[column]
    escalation_rate[axis] = df[axis]
    return escalation_rate.drop(columns=[column])


In [ ]:
def standardize_format(
    df,
    param,
    classLabel1,
    units,
    valueColumn,
    class1 = 'n/a',
    class1Column = None,
    vintageColumn='vintage',
    xColumn='year',
    xLabel='Year',
    classLabel2='technology',
    class2Column='technology',
    subRegionColumn='region',
    region='USA',
    scenario=f'Main_database_basexdb_{gcam_scenario}',
) -> pd.DataFrame:
    clone = df.copy()
    if vintageColumn in clone.columns:
        clone['vintage'] = 'Vint_' + clone[vintageColumn].astype(int).astype(str)
    else:
        clone['vintage'] = 'Vint_2015'
    clone = clone.rename(columns={
        class2Column: 'class2',
        subRegionColumn: 'subRegion',
        valueColumn: 'value',
        xColumn: 'x',
    })
    if not 'x' in clone.columns:
        clone['x'] = 2015
    clone['scenario'] = scenario
    clone['region'] = region
    clone['xLabel'] = xLabel
    clone['classLabel2'] = classLabel2
    if class1Column is not None:
        clone['class1'] = clone[class1Column]
    else:
        clone['class1'] = class1
    clone['classLabel1'] = classLabel1
    clone['param'] = param
    clone['units'] = units
    clone['x'] = clone['x'].astype(int)
    return clone[['scenario', 'region', 'subRegion', 'param', 'classLabel1', 'class1', 'classLabel2', 'class2', 'xLabel', 'x', 'vintage', 'units', 'value']].reset_index(drop=True)

## Capacity

##### Generation

In [ ]:
# capacity will need to be calculated from generation and capacity factor
generation = conn.runQuery(
    get_query_by_name(queries, generation_query_name),
    regions=[
        # TODO need to use this
    ]
)
generation['vintage'] = generation['technology'].str.slice(-4).astype(int)
generation['technology'] = generation['technology'].str.slice(0, -10)
generation['subsector.1'] = generation['subsector.1'].str.slice(0, -8)
# convert EJ to GW
generation['value'] = generation['value'] * EXAJOULES_TO_GIGAWATTS
generation = generation.drop(columns=['subsector']).rename(columns={
    'Year': 'year',
    'Units': 'units',
    'value': 'generation_GW',
    'subsector.1': 'subsector',
})
generation = generation[[
    'year', 'vintage', 'region', 'sector', 'subsector', 'technology', 'generation_GW'
]].reset_index(drop=True)

In [ ]:
generation#[(generation.region == 'CA') & (generation.year != generation.vintage)].technology.unique()

##### Capacity Factor

In [ ]:
# capacity factor by subsector and technology
# some techs are for all regions and some are state specific
capacity_factor_conventional = pd.read_csv(capacity_factor_conventional_file, comment='#')[['sector.name', 'subsector.name', 'year', 'capacity.factor', 'technology']]
capacity_factor_conventional['region'] = [ALL_STATES for i in capacity_factor_conventional.index]
capacity_factor_conventional = capacity_factor_conventional.explode('region', ignore_index=True)
capacity_factor_conventional.rename(columns={
    'sector.name': 'supplysector',
    'subsector.name': 'subsector',
}, inplace=True)
capacity_factor_wind = pd.read_csv(capacity_factor_wind_file, comment='#')[['supplysector', 'subsector', 'year', 'capacity.factor', 'technology', 'region']]
capacity_factor_pv = pd.read_csv(capacity_factor_pv_file, comment='#')[['supplysector', 'subsector', 'year', 'capacity.factor', 'technology', 'region']]
capacity_factor_csp = pd.read_csv(capacity_factor_csp_file, comment='#')[['supplysector', 'subsector', 'year', 'capacity.factor', 'technology', 'region']]

# the above renewable capacity factors are derived using oversimplified assumptions,
# so we will replace them with the NREL assumptions
nrel_renewables_capacity_factors = pd.read_csv(nrel_renewables_capacity_factor_file, comment='#')
nrel_renewables_capacity_factors['State'] = nrel_renewables_capacity_factors['State'].str.lower().map({v.replace('_', ' ').lower(): k for k, v in STATE_ABBREVIATIONS_TO_NAMES.items()})

# correct MS onshore by averaging nearby states
nrel_renewables_capacity_factors.loc[
    nrel_renewables_capacity_factors.index[nrel_renewables_capacity_factors.State == 'MS'],
    'Onshore_Wind'
] = nrel_renewables_capacity_factors[
    nrel_renewables_capacity_factors.State.isin(['LA', 'AL', 'AR'])
]['Onshore_Wind'].mean()

# correct AL offshore by averages nearby states
nrel_renewables_capacity_factors.loc[
    nrel_renewables_capacity_factors.index[nrel_renewables_capacity_factors.State == 'AL'],
    'Offshore_Wind'
] = nrel_renewables_capacity_factors[
    nrel_renewables_capacity_factors.State.isin(['MS', 'FL',])
]['Offshore_Wind'].mean()

# urban and rural utility scale PV appear to be the same for all WECC states, so just using urban here
# divide by 1.3 to convert from AC to DC
nrel_renewables_capacity_factors['Urban_Utility_scale_PV'] = nrel_renewables_capacity_factors['Urban_Utility_scale_PV'] / 1.3
capacity_factor_pv['capacity.factor'] = capacity_factor_pv[['region']].merge(
    nrel_renewables_capacity_factors[['State', 'Urban_Utility_scale_PV']], how='left', left_on='region', right_on='State'
)['Urban_Utility_scale_PV'].values

capacity_factor_csp['capacity.factor'] = capacity_factor_csp[['region']].merge(
    nrel_renewables_capacity_factors[['State', 'CSP']], how='left', left_on='region', right_on='State'
)['CSP'].values

# for wind, need to split by onshore and offshore
capacity_factor_wind_onshore = capacity_factor_wind[~capacity_factor_wind['technology'].str.contains('offshore')].copy()
capacity_factor_wind_onshore['capacity.factor'] = capacity_factor_wind_onshore[['region']].merge(
    nrel_renewables_capacity_factors[['State', 'Onshore_Wind']], how='left', left_on='region', right_on='State'
)['Onshore_Wind'].values
capacity_factor_wind_offshore = capacity_factor_wind[capacity_factor_wind['technology'].str.contains('offshore')].copy()
capacity_factor_wind_offshore['capacity.factor'] = capacity_factor_wind_offshore[['region']].merge(
    nrel_renewables_capacity_factors[['State', 'Offshore_Wind']], how='left', left_on='region', right_on='State'
)['Offshore_Wind'].values

capacity_factors = pd.concat([
    capacity_factor_conventional[~capacity_factor_conventional.technology.str.contains('base_storage')],
    capacity_factor_wind_onshore, capacity_factor_wind_offshore,
    capacity_factor_pv, capacity_factor_csp,
])

capacity_factors.rename(columns={
    'supplysector': 'sector',
    'capacity.factor': 'capacity_factor',
}, inplace=True)

capacity_factors = capacity_factors[['year', 'region', 'sector', 'subsector', 'technology', 'capacity_factor']].reset_index(drop=True)

In [ ]:
capacity_factors#[capacity_factors.technology.str.contains('wind') & (capacity_factors.region.isin(['MS', 'AL'])) & (capacity_factors.capacity_factor == 0)]# & (capacity_factors.year == 2025)]

##### Capacity

In [ ]:
# first join where region is all
all_regions = generation.merge(capacity_factors[capacity_factors.region == '*'][['year', 'sector', 'subsector', 'technology', 'capacity_factor']], how='left', on=['year', 'sector', 'subsector', 'technology'])
# then join on states specifically
state_regions = generation.merge(capacity_factors[capacity_factors.region != '*'][['year', 'region', 'sector', 'subsector', 'technology', 'capacity_factor']], how='left', on=['year', 'region', 'sector', 'subsector', 'technology'])
capacity = state_regions.combine_first(all_regions)
del all_regions
del state_regions
# HACK for hydro and other missing capacity factors TODO fix it?
capacity.fillna({'capacity_factor': 0.38}, inplace=True)
capacity['capacity_GW'] = (capacity['generation_GW'] / capacity['capacity_factor']) # TODO what's the right way to use cap factor if at all?
# sum generation and capacity over sectors... TODO is this correct?
capacity = capacity.drop(columns=['sector']).groupby(['year', 'vintage', 'region', 'technology']).aggregate({
    'generation_GW': 'sum',
    #'capacity_factor': 'first',
    'capacity_GW': 'sum',
}).reset_index()
capacity['year'] = capacity['year'].astype(int)
capacity['vintage'] = capacity['vintage'].astype(int)
capacity = pd.concat([
    standardize_format(capacity, param='elec_cap_usa_GW', classLabel1='subsector', units='GW', valueColumn='capacity_GW'),
    #standardize_format(capacity, param='elec_gen_usa_GW', classLabel1='subsector', units='GW', valueColumn='generation_GW'),
    standardize_format(capacity_factors, param='elec_capacity_factor_usa_in', classLabel1='investment_segment', class1Column='sector', units='Capacity Factor', valueColumn='capacity_factor', vintageColumn='year'),
])


In [ ]:
capacity#[capacity.class2.str.contains('offshore') & (capacity.param == 'elec_capacity_factor_usa_in') & (capacity.x == 2035)]

## Heat Rate

In [ ]:
# heat rate is based on technology and input fuel
# TODO something is wrong with heat rate vintage
heat_rate = conn.runQuery(get_query_by_name(queries, heat_rate_query_name))
heat_rate['vintage'] = heat_rate['IO-coefficient'].str.slice(-4).astype(int)
# convert to BTU per kWh
# what unit is the original??
heat_rate['value'] = heat_rate['value'] * BRITISH_THERMAL_UNITS_PER_KILOWATT_HOUR
heat_rate.rename(columns={
    'sector.1': 'technology',
    'Units': 'units',
    'Year': 'year',
    'input': 'fuel_type',
    'value': 'heat_rate_BTUperkWh',
}, inplace=True)
heat_rate = heat_rate[(~heat_rate.fuel_type.str.contains('backup|credits|water'))][[
    'year', 'vintage', 'region', 'technology', 'fuel_type', 'heat_rate_BTUperkWh'
]].reset_index(drop=True)
# use this to create a mapping of technology to fuel types
tech_to_fuel = heat_rate[['technology', 'fuel_type']].copy().drop_duplicates(ignore_index=True)
# transform to expected format
heat_rate = standardize_format(heat_rate, param='elec_heat_rate_BTUperkWh', classLabel1='subsector', class1Column='fuel_type', units='Heat Rate (BTU per kWh)', valueColumn='heat_rate_BTUperkWh')


In [ ]:
heat_rate[(heat_rate.x == 2035) & (heat_rate.subRegion == 'WA')]

## Fuel Prices

In [ ]:
# fuel prices are provided for biomass per state and for fossils per grid region
prices = conn.runQuery(get_query_by_name(queries, prices_query_name))
# convert 1975$ to 2015$ following gcamextractor gdp_deflator
# convert GJ to MBTU
prices['value'] = prices['value'] * deflate_gdp(2015, 1975) / GIGAJOULES_TO_MEGA_BRITISH_THERMAL_UNITS
prices.rename(columns={
    'sector': 'fuel_type',
    'value': 'fuel_price_2015USDperMBTU',
    'Year': 'year',
}, inplace=True)
prices = prices[prices.fuel_type.isin(['regional biomass', 'refined liquids industrial', 'regional coal', 'wholesale gas'])][[
    'year', 'region', 'fuel_type', 'fuel_price_2015USDperMBTU'
]].reset_index(drop=True)
# for biomass only, map USA to all states
# for the other fuels, map grid region to state
bio = prices[(prices['fuel_type'] == 'regional biomass') & (prices['region'] == 'USA')].copy()
bio['region'] = [ALL_STATES for i in bio.index]
bio = bio.explode('region', ignore_index=True)
prices = pd.concat([
    bio,
    grid_region_to_state(prices).sort_values(['region', 'fuel_type', 'year']).reset_index(drop=True),
]).reset_index(drop=True)
# fuel price escalation rate is (<price> - <price last period>) / <price last period>
price_escalation_rate = get_escalation_rate(prices, 'fuel_price_2015USDperMBTU', ['region', 'fuel_type'], 'fuel_price_escalation_rate_fraction')
# expand to technologies
prices = tech_to_fuel.merge(prices, how='left', on='fuel_type')
prices = prices[prices.region.notna()]
price_escalation_rate = tech_to_fuel.merge(price_escalation_rate, how='left', on='fuel_type')
price_escalation_rate = price_escalation_rate[price_escalation_rate.region.notna()]
# transform to expected format
prices = standardize_format(prices, param='elec_fuel_price_2015USDperMBTU', classLabel1='fuel', class1Column='fuel_type', units='Fuel Cost (2015 USD/MBTU)', valueColumn='fuel_price_2015USDperMBTU', vintageColumn='year')
price_escalation_rate = standardize_format(price_escalation_rate, param='elec_fuel_price_escl_rate_fraction', classLabel1='fuel', class1Column='fuel_type', units='Fuel Price Escalation Rate (fraction)', valueColumn='fuel_price_escalation_rate_fraction', vintageColumn='year')


In [ ]:
prices

In [ ]:
price_escalation_rate

## Fixed and Variable O&M Costs

##### Fixed O&M

In [ ]:
fixed_operation_and_maintenance_cost = [
    pd.read_csv(f, comment='#') for f in fixed_operation_and_maintenance_cost_files
]
for i, df in enumerate(fixed_operation_and_maintenance_cost):
    # the original unit 1975$/kW-year
    df['OM.fixed'] = df['OM.fixed'] * deflate_gdp(2015, 1975) * 1000 / 8760
    df.rename(columns={
        'intermittent.technology': 'technology',
        'OM.fixed': 'fixed_om_2015USDperMWh'
    }, inplace=True)
    fixed_operation_and_maintenance_cost[i] = df[['year', 'technology', 'fixed_om_2015USDperMWh']].drop_duplicates(ignore_index=True).reset_index(drop=True)
fixed_operation_and_maintenance_cost = pd.concat(fixed_operation_and_maintenance_cost).sort_values(['technology', 'year']).reset_index(drop=True)
# expand to all states
fixed_operation_and_maintenance_cost['region'] = [ALL_STATES for i in fixed_operation_and_maintenance_cost.index]
fixed_operation_and_maintenance_cost = fixed_operation_and_maintenance_cost.explode('region', ignore_index=True)
# transform to expected format
fixed_operation_and_maintenance_cost = standardize_format(fixed_operation_and_maintenance_cost, param='elec_fixed_om_2015USDperMWh', classLabel1='subsector', units='Fixed OnM Cost (2015 USD/MWh)', valueColumn='fixed_om_2015USDperMWh', vintageColumn='year')


In [ ]:
fixed_operation_and_maintenance_cost

##### Variable O&M

In [ ]:
variable_operation_and_maintenance_cost = [
    pd.read_csv(f, comment='#') for f in variable_operation_and_maintenance_cost_files
]
# used to expand intermittent technologies to all subtypes
tech_mapping = pd.read_csv(tech_mapping_file, comment='#')[['Electric.sector.technology', 'to.technology']]
for i, df in enumerate(variable_operation_and_maintenance_cost):
    # original unit 1975$/MWh
    df['OM.var'] = df['OM.var'] * deflate_gdp(2015, 1975)
    if 'intermittent.technology' in df.columns:
        # need to expand the intermittent technologies to include all the subtypes
        df = df.merge(tech_mapping, how='left', left_on='intermittent.technology', right_on='Electric.sector.technology').rename(columns={
            'to.technology': 'technology'
        })
    df.rename(columns={
        'OM.var': 'variable_om_2015USDperMWh'
    }, inplace=True)
    variable_operation_and_maintenance_cost[i] = df[['technology', 'year', 'variable_om_2015USDperMWh']].drop_duplicates(ignore_index=True).reset_index(drop=True)
variable_operation_and_maintenance_cost = pd.concat(variable_operation_and_maintenance_cost).sort_values(['technology', 'year']).reset_index(drop=True)
# variable o&m escalation rate is (<var o&m> - <var o&m last period>) / <var o&m last period>
variable_operation_and_maintenance_cost_escalation_rate = get_escalation_rate(variable_operation_and_maintenance_cost, 'variable_om_2015USDperMWh', ['technology'], 'variable_om_escl_rate_fraction')
# expand to all regions
variable_operation_and_maintenance_cost['region'] = [ALL_STATES for i in variable_operation_and_maintenance_cost.index]
variable_operation_and_maintenance_cost = variable_operation_and_maintenance_cost.explode('region', ignore_index=True)
variable_operation_and_maintenance_cost_escalation_rate['region'] = [ALL_STATES for i in variable_operation_and_maintenance_cost_escalation_rate.index]
variable_operation_and_maintenance_cost_escalation_rate = variable_operation_and_maintenance_cost_escalation_rate.explode('region', ignore_index=True)
# transform to expected columns
variable_operation_and_maintenance_cost = standardize_format(variable_operation_and_maintenance_cost, param='elec_variable_om_2015USDperMWh', classLabel1='subsector', units='Variable OnM Cost (2015 USD/MWh)', valueColumn='variable_om_2015USDperMWh', vintageColumn='year')
variable_operation_and_maintenance_cost_escalation_rate = standardize_format(variable_operation_and_maintenance_cost_escalation_rate, param='elec_variable_om_escl_rate_fraction', classLabel1='subsector', units='Variable OnM Cost Escalation Rate (fraction)', valueColumn='variable_om_escl_rate_fraction', vintageColumn='year')


In [ ]:
variable_operation_and_maintenance_cost

In [ ]:
variable_operation_and_maintenance_cost_escalation_rate

## CO2 Content

In [ ]:
carbon_content = [
    pd.read_csv(f, comment='#') for f in carbon_content_files
]
for i, df in enumerate(carbon_content):
    if not 'WA' in df['region'].values:
        df = df[df['region'] == 'USA'].drop_duplicates(ignore_index=True)
        # expand to all states
        df['region'] = [ALL_STATES for i in df.index]
        df = df.explode('region', ignore_index=True)
    # TODO is this conversion correct? what's going on here? Zarrar: original units are kg carbon / GJ
    df['PrimaryFuelCO2Coef'] = df['PrimaryFuelCO2Coef'] * ATOMIC_WEIGHT_OF_CO2_PER_ATOMIC_WEIGHT_OF_CARBON * TONS_PER_KG / (MEGAWATT_HOURS_PER_GIGAJOULE * MEGA_BRITISH_THERMAL_UNITS_PER_MEGAWATT_HOUR)
    df.rename(columns={
        'PrimaryFuelCO2Coef': 'fuel_co2_content_tonsperMBTU',
        'PrimaryFuelCO2Coef.name': 'fuel_type'
    }, inplace=True)
    carbon_content[i] = df[['region', 'fuel_type', 'fuel_co2_content_tonsperMBTU']].reset_index(drop=True)
carbon_content = pd.concat(carbon_content).reset_index(drop=True)
# remove remaining USA entries
carbon_content = carbon_content[carbon_content['region'] != 'USA']
# expand to technologies
carbon_content = tech_to_fuel.merge(carbon_content, how='left', on='fuel_type')
carbon_content = carbon_content[carbon_content.region.notna()]
# transform to expected format
carbon_content = standardize_format(carbon_content, param='elec_fuel_co2_content_tonsperMBTU', classLabel1='fuel', class1Column='fuel_type', units='Fuel CO2 Content (Tons per MBTU)', valueColumn='fuel_co2_content_tonsperMBTU')
# repeat the rows for each half-decade
for y in np.arange(2020, 2105, 5):
    copy = carbon_content[carbon_content.x == 2015].copy()
    copy['x'] = y
    copy['vintage'] = f'Vint_{y}'
    carbon_content = pd.concat([carbon_content, copy])

In [ ]:
carbon_content#[(carbon_content.subRegion == 'AZ') & (carbon_content.x == 2020) & (carbon_content.class2 == 'coal (conv pul) (once through)')]

## CO2 Capture

In [ ]:
carbon_capture = pd.read_csv(carbon_capture_file, comment='#')
carbon_capture.rename(columns={
    'remove.fraction': 'carbon_capture_rate_fraction',
}, inplace=True)
carbon_capture = carbon_capture[['year', 'technology', 'carbon_capture_rate_fraction']].reset_index(drop=True)
# expand to all states
carbon_capture['region'] = [ALL_STATES for i in carbon_capture.index]
carbon_capture = carbon_capture.explode('region', ignore_index=True)
# transform to expected format
carbon_capture = standardize_format(carbon_capture, param='elec_carbon_capture_rate_fraction', classLabel1='subsector', units='Carbon Capture Rate (fraction)', valueColumn='carbon_capture_rate_fraction', vintageColumn='year')

In [ ]:
carbon_capture

## Lifetime and Lifetime Parameters

##### Lifetime years

In [ ]:
lifetime = [
    pd.read_csv(f, comment='#') for f in lifetime_files
]
for i, df in enumerate(lifetime):
    df.rename(columns={
        'lifetime': 'lifetime_yr',
        'intermittent.technology': 'technology'
    }, inplace=True)
    lifetime[i] = df[['year', 'technology', 'lifetime_yr']].reset_index(drop=True)
lifetime = pd.concat(lifetime).reset_index(drop=True)
# expand to all states
lifetime['region'] = [ALL_STATES for i in lifetime.index]
lifetime = lifetime.explode('region', ignore_index=True)
# transform to expected format
lifetime = standardize_format(lifetime, param='elec_lifetime_yr', classLabel1='subsector', units='lifetime (yr)', valueColumn='lifetime_yr', vintageColumn='year').drop_duplicates()

In [ ]:
lifetime#[(lifetime.subRegion == 'AZ') & (lifetime.x == 2020) & (lifetime.class2 == 'coal (conv pul) (once through)')]

### Combine everything and write to file

In [ ]:
output = pd.concat([
    capacity,
    heat_rate,
    prices,
    price_escalation_rate,
    fixed_operation_and_maintenance_cost,
    variable_operation_and_maintenance_cost,
    variable_operation_and_maintenance_cost_escalation_rate,
    carbon_content,
    carbon_capture,
    lifetime,
])
# filter to WECC states
#output = output[output.subRegion.isin(['CA', 'OR', 'WA', 'AZ', 'NV', 'WY', 'ID', 'UT', 'NM', 'CO', 'MT'])]
#output = output[output.vintage.str.slice(-4).astype(int) >= 2020]

In [ ]:
output[(output.param == 'elec_cap_usa_GW') & (output.subRegion == 'WA') & (output.class2.str.contains('CCS'))]

In [ ]:
output.to_csv(f'{gcam_prefix}{gcam_scenario}_DataTable.csv', index=False)